In [1]:
%load_ext autoreload

## Spectrometer Control Notebook

This is the GUI version. All helper routines have been moved to the file helpers/Spectrometer.py and get imported in the import statement below 📈. We will need one more module. Open a terminal and enter

```
sudo pip3 install ipympl
```

In [2]:
from helpers.Spectrometer import *

plt.rcParams['figure.figsize'] = [10, 7]
%matplotlib widget

%autoreload 2

### Setup

We now create an instance of the spectrometer class. The code is in `helpers/Spectrometer.py` imported earlier. Feel free to check it out 😎

In [3]:
spec = Spectrometer(lcd=True, neopixel=True)

Now we set the default values. Note you can change many of them in the spectrometer interface as well.

In [4]:
# Local web log server (run python3 -m http.server) and video stream
spec.localurl  = 'http://orcspi.local:8000/'
spec.streamurl  = 'http://orcspi.local:4664/'

# Other defaults
spec.m_name.value = "Thomas P."

# Rotation and cropping
spec.p_rot.value  = str(0.0)
spec.setCrop([0,200,250,290])

# Calibration line defaults (here CFL bulb) 
spec.p_pix1.description = "Pix Green"
spec.p_pix1.value = "122"
spec.waveL1 = 544.0

spec.p_pix2.description = "Pix Orange"
spec.p_pix2.value = "180"
spec.waveL2 = 611.0

Finally here we bring up the spectrometer interface. Happy measuring 📈

In [6]:
spec.show()

#### Playground

In [ ]:
spec.close()